In [1]:
import pandas as pd
from selenium import webdriver
from datetime import datetime
import re
import pytz
from Operations import ReadSheet

## Import line below for Linux
#from pyvirtualdisplay import Display

# Linux Must install sudo apt-get install xvfb, pip install xvfbwrapper 

readSheet=ReadSheet()
sheetFlow=readSheet.Authorization_Flow()

monthDict={'ม.ค.':'01',
           'ก.พ.':'02',
           'มี.ค.':'03',
           'เม.ย.':'04',
           'พ.ค.':'05',
           'มิ.ย.':'06',
           'ก.ค.':'07',
           'ส.ค.':'08',
           'ก.ย.':'09',
           'ต.ค.':'10',
           'พ.ย.':'11',
           'ธ.ค.':'12'    
}

url='https://marketdata.set.or.th/mkt/investortype.do?language=th&country=TH'

def Authorization(url):
    ## These two lines for Linux
    #display=Display(visible=0, size=(1024,768))
    #display.start()
    
    option=webdriver.ChromeOptions()
    option.add_argument("--incognito")
    driver = webdriver.Chrome(chrome_options=option)
    driver.implicitly_wait(30)
    driver.get(url)
    return driver #, display

def TerminateConnection(driver): #,display):
    driver.quit()
    ## Line below for Linux
    #display.stop()

def GetCurrentDate(driver):
    element = driver.find_elements_by_tag_name('strong')

    #for title in element:
        #output=title.text
        #print(output, ' == ',type(output))

    #print(type(element))
    dateFirst=element[1].text
    locFirst=dateFirst.index(r"วันที่")

    dateFirst_mod=re.split('\s+',dateFirst[locFirst:])

    year=int(dateFirst_mod[3])-543
    month=int(monthDict[dateFirst_mod[2]])
    day=int(dateFirst_mod[1]) 
    currentDate=datetime(year,month,day )
    
    
    currentString=currentDate.strftime("%Y-%m-%d")

    return currentDate, currentString

def GetPriceValue(driver):    
    element = driver.find_elements_by_tag_name('td')
    lenCurrent=(7*4)
    elements=element[:lenCurrent]

    eleList=[]
    for title in elements:
        #output=title.text
        eleList.append(title.text)
    #print(output, ' == ',type(output))
    
    is_data=list(zip(*[iter(eleList)]*7))
    #print(' isd : ', is_data)
           
    flowDf=pd.DataFrame(is_data[0:])
    flowDf.columns=['category','Buy_val','Buy_percent','Sell_val','Sell_percent','Net_val','Net_percent']
    flowDf_1=flowDf[['category','Net_val']].copy()
    flowDf_T=flowDf_1.T
    flowDf_T.columns=flowDf_T.iloc[0]
    flowDf_T=flowDf_T.iloc[1:,]
    flowDict=flowDf_T.to_dict()
    valOut=[]
    for n in list(flowDict):
        s=flowDict[n]['Net_val']
        s = float(s.replace(',', ''))        
        valOut.append(s)
    
    return   flowDf_T, valOut
  
   
driver=Authorization(url)    
# Linux version
#driver, display =Authorization(url)    

currentDate, currentString=GetCurrentDate(driver)
#print(' date : ',currentString)    

flowDf, valOut=GetPriceValue(driver)
#print(' ==> ',flowDf)

todayStr, nowDate, nowTime = readSheet.GetDateTime()
readSheet.InsertNewValue_Flow(todayStr, nowDate, nowTime, sheetFlow[0], currentString, valOut[0], valOut[1], valOut[2], valOut[3])


TerminateConnection(driver) #,display)

C:\Users\70018928\AppData\Local\Continuum\anaconda3\envs\quandl\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
C:\Users\70018928\AppData\Local\Continuum\anaconda3\envs\quandl\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: use options instead of chrome_options


 date :  2020-04-30
 ==>  category สถาบันในประเทศ บัญชีบริษัทหลักทรัพย์ นักลงทุนต่างประเทศ  \
Net_val        4,975.10                 82.06          -2,159.15   

category นักลงทุนทั่วไปในประเทศ  
Net_val               -2,898.02  
 len :  2
 lastDate :  2020-04-30
Updated at  00:31:56
